<a href="https://colab.research.google.com/github/anhbaysgalan1/1stsem20192020/blob/master/Mongolian_text_classification_01%2C_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mongolian text classification series #01

In this notebook I'm gonna try to classify cyrillic mongolian texts using modern Tensorflow 2.0

Eduge dataset provided by Bolorsoft LLC

Author : Sharavsambuu Gunchinish (sharavsambuu@gmail.com)

Github: https://github.com/sharavsambuu/mongolian-text-classification 



In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow==2.0.0-beta1 
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

     |████████████████████████████████| 87.9MB 1.2MB/s 
     |████████████████████████████████| 3.1MB 32.9MB/s 
     |████████████████████████████████| 501kB 52.9MB/s 
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1
2.0.0-beta1


[More info about creation of eduge dataset pickles](https://github.com/sharavsambuu/mongolian-text-classification/blob/master/preprocess_dataset/preprocess_eduge.ipynb) 

In [0]:
import os
from os.path import exists, join, basename, splitext
import sys

def download_from_google_drive(file_id, file_name):
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=$file_id" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=$confirm_text&id=$file_id" -o $file_name
  
# download eduge pickles
file_path = 'eduge_pickles'
if not exists(file_path):
  download_from_google_drive('1vjJ9YgIe8o0ErhbN0lH1XqPv3KFP8acv', '%s.rar' % file_path)
  rar_file = file_path+".rar"
  !unrar x $rar_file

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   4619      0 --:--:-- --:--:-- --:--:--  4619
100  106M    0  106M    0     0  47.5M      0 --:--:--  0:00:02 --:--:-- 58.9M

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from eduge_pickles.rar

Extracting  word_index.pickle                                              2%  OK 
Extracting  eduge.pickle                                                   6% 10% 14% 17% 21% 25% 29% 32% 36% 40% 44% 47% 51%  OK 
Extracting  eduge_stopwords_removed.pickle                                55% 58% 62% 66% 70% 73% 77% 81% 85% 88% 92% 96% 97%  OK 
Extracting  reversed_word_index.pickle                                    99%  OK 
All OK


In [0]:
import pickle

with open('word_index.pickle', 'rb') as handle:
  word_index = pickle.load(handle)
    
with open('reversed_word_index.pickle', 'rb') as handle:
  reversed_word_index = pickle.load(handle)
  
with open('eduge_stopwords_removed.pickle', 'rb') as handle:
  eduge_ds_raw = pickle.load(handle)

In [0]:
#MAX_LEN = 512
#import itertools
#for item in eduge_ds:
#  item[0] = list(itertools.chain(*item[0]))[:MAX_LEN]

MAX_LEN = 512

import itertools
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
labels_list = [l[1] for l in eduge_ds_raw] # collecting labels
onehots = encoder.fit_transform(labels_list)

eduge_ds = []
for idx, item in enumerate(eduge_ds_raw):
  sentence = list(itertools.chain(*item[0]))[:MAX_LEN]
  eduge_ds.append((sentence, onehots[idx], item[1]))

In [0]:
print(eduge_ds[0][2])
print(eduge_ds[0][1])
print(encoder.classes_)

урлаг соёл
[0 0 0 0 0 1 0 0 0]
['байгал орчин' 'боловсрол' 'спорт' 'технологи' 'улс төр' 'урлаг соёл'
 'хууль' 'эдийн засаг' 'эрүүл мэнд']


In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(eduge_ds, test_size=0.1, random_state=999)

In [0]:
train_data_words    = [i[0] for i in train]
train_label_onehots = [i[1] for i in train]

test_data_words     = [i[0] for i in test ]
test_label_onehots  = [i[1] for i in test ]

In [0]:
def encode_news(text):
    return [word_index.get(i, 2) for i in text]
  
train_data = [encode_news(sent) for sent in train_data_words]
test_data  = [encode_news(sent) for sent in test_data_words ]

In [0]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=MAX_LEN)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=MAX_LEN)

In [0]:
train_label_onehots[:3]

[array([0, 0, 0, 1, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0])]

In [0]:
vocab_size = len(word_index)

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(len(encoder.classes_), activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          5932704   
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 153       
Total params: 5,933,129
Trainable params: 5,933,129
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
partial_index   = 3000

partial_x_train = train_data[partial_index:]
partial_y_train = np.array([np.array(l) for l in train_label_onehots[partial_index:]])

x_val           = train_data[:partial_index]
y_val           = np.array([np.array(l) for l in train_label_onehots[:partial_index]])


In [0]:
epochs = 30
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

W0618 16:26:34.552035 140085617461120 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 65094 samples, validate on 3000 samples
Epoch 1/30
65094/65094 [==============================] - 14s 210us/sample - loss: 2.1482 - accuracy: 0.1817 - val_loss: 2.0865 - val_accuracy: 0.2047
Epoch 2/30
65094/65094 [==============================] - 13s 205us/sample - loss: 1.9897 - accuracy: 0.3142 - val_loss: 1.8662 - val_accuracy: 0.3443
Epoch 3/30
65094/65094 [==============================] - 13s 207us/sample - loss: 1.6907 - accuracy: 0.4206 - val_loss: 1.5172 - val_accuracy: 0.4797
Epoch 4/30
65094/65094 [==============================] - 13s 205us/sample - loss: 1.3200 - accuracy: 0.5658 - val_loss: 1.1784 - val_accuracy: 0.6413
Epoch 5/30
65094/65094 [==============================] - 13s 205us/sample - loss: 1.0342 - accuracy: 0.7035 - val_loss: 0.9725 - val_accuracy: 0.7313
Epoch 6/30
65094/65094 [==============================] - 13s 205us/sample - loss: 0.8617 - accuracy: 0.7700 - val_loss: 0.8508 - val_accuracy: 0.7840
Epoch 7/30
65094/65094 [=====================

In [0]:
test_label_onehots = np.array([np.array(l) for l in test_label_onehots])

In [0]:
results = model.evaluate(test_data, test_label_onehots)
print(results)

7567/7567 [==============================] - 2s 329us/sample - loss: 0.4172 - accuracy: 0.8944
[0.4172078494026369, 0.89440995]


In [0]:
def softmax(w, t = 1.0):
  e = np.exp(np.array(w) / t)
  dist = e / np.sum(e)
  return dist

In [0]:
# learned data
data_index   = 23
data_words   = " ".join(train_data_words[data_index])
data_indexes = train_data[data_index]
y_pred = model.predict([data_indexes])
print(data_words)
print(encoder.classes_[np.argmax(softmax(y_pred[0]))])

хаврын чуулган завсарласнаас хойш багагүй хугацаанд амралтын байдалтай уихын үйл ажиллагаа жигдэрч эхлэх гишүүд амралтаа дуусгаж сарын эхлэн ажилдаа орохоор болоод байгаа зарим гишүүдийн хувьд тойрогтоо ажиллаж байгаа уулзалт хурал цуглаан зочид төлөөлөгчид гэх мэтээр өдөр болгон ажилтай байгаа тухайлбал уих дахь анын бүлгийн дарга ажилдаа орж сэлэнгэ аймгийн мандал суманд ажилласан бөгөөд тун удахгүй хөдөө нутгаар ажиллахаар болсон гэнэ тэгвэл уихын дарга өчигдрөөс эхлэн ажилдаа орж байгаа долдугаар сарын ээлжийн амралтаа аваад уихын дарга зэнхболд ажилдаа орсноор уихын үйл ажиллагаа жигдэрч эхлэх шинэчлэлийн засгийн газрын сайдууд болон ерөнхий сайдад амрах хугацаа олдохгүй байгаа бөгөөд засгийн газрын хуралдаанаа булган аймагт хуралдуулж хоёр хөршийн төрийн тэргүүний айлчлалд анхаарлаа хандуулж байгаа наадмаар амрахгүй ажиллана хэмээн мэдэгдэж манын бүлэг одоогоор ажилдаа ороогүй амралтын байдалтай манын ерөнхий нарийн бичгийн дарга жмөнхбат ээлжит бус чуулган хуралдуулах мэдэгдэл г

In [0]:
# unseen data
data_index   = 13
data_words   = " ".join(test_data_words[data_index])
data_indexes = test_data[data_index]
print(data_words)
y_pred = model.predict([data_indexes])
print(encoder.classes_[np.argmax(softmax(y_pred[0]))])

өвөрмонголын газрын ховор элементийн үйлдвэрлэлд хятад квот тогтоосон талаар тус улсын газрын эрдэс баялгийн яам өчигдөр мэдэгдсэн тус яамны орон нутгийн салбараас явуулсан мэдэгдэлд өвөрмонгол дахь төрийн өмчийн төмөр гангийн үйлдвэрлэгч baotou steel group хятад мянган тоннын экспортын квот тогтоосон гэнэ тус групп хятадын газрын ховор элементийн хамгийн том үйлдвэрлэгч baotou steel rareearth hitech эцэг компани дээрх хоёр компани хоёулаа өвөрмонголд үйлдвэрлэл явуулдаг аж тус групп онд тонн бүтээгдэхүүн үйлдвэрлэсэн дангаараа дэлхийн газрын ховор элементийн хувийг нийлүүлдэг хамгийн том үйлдвэрлэгч хятад улс онд гэхэд экспортын бүтээгдэхүүнд тоннын квот тогтоосон хятад оноос хойш газрын ховор элементийн экспортыг хязгаарлаж квот тогтоох болсноос хойш төрлийн түүхий эдийн үнэ дэлхийн зах эрс нэмэгдэж байгаа билээ нэнх
урлаг соёл
